In [1]:
import os
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from composio_langchain import ComposioToolSet, App
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool
from langchain_core.tools import ToolException
from typing import List, Optional, Dict, Any
import warnings

warnings.filterwarnings("ignore")

/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/Users/morossini/anaconda3/envs/composio/lib/python3.12/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = ChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.enhance_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to. This is a core instruction: ALWAYS ALWAYS ALWAYS USE A STRING AS ID"),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")

    def try_except_tool(self, func):
        try:
            return func
        except ToolException as e:
            print(f"Error: {e}")

    def enhance_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self.try_except_tool(tool.func),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [3]:
composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))
agent_tools = composio_toolset.get_tools(apps=[App.CLICKUP])

KeyError: 'null'

In [ ]:
for i, tool in enumerate(agent_tools):
    print(f"{tool.name.upper},")

In [ ]:
print(f"Tool name: {agent_tools[61].name}\n")
print("Function details:")
agent_tools[61].func

In [ ]:
agent = MyAgent(tools=agent_tools)

In [ ]:
agent.myinvoke("Retrieve the name of my clickup workspace")

In [ ]:
clickup_team_id="12927880"

In [ ]:
agent.myinvoke(f"Which tools do you have for working with Clickup spaces?")

In [ ]:
agent.myinvoke(f"Use 'clickup_spaces_get_space_details' with the following team_id to view available Spaces in my Workspace. team_id: {clickup_team_id}")

In [ ]:
agent.myinvoke("Use 'clickup_spaces_get_space_details' with the following team_id to view available Spaces in my Workspace. team_id:'12927880'. You MUST pass the team_id as an integer, NOT as a float or as a string")

In [ ]:
agent.myinvoke("What can you tell me about my Clickup space of 54804921?")

In [ ]:
agent.myinvoke("Can you give me a list of Spaces in my clickup workspace?")

In [ ]:
agent.myinvoke("Retrieve a list of members in my workspace")

In [ ]:
agent_tools[50]

In [ ]:
agent_tools[0].func

In [ ]:
type(agent_tools)

In [ ]:
for i, tool in enumerate(agent_tools):
    if "clickup_lists_get_list_details" in tool.name:
        pprint(f"{i}: {tool}")

In [ ]:
import inspect
from pprint import pprint

# Retrieve the Pydantic model class for the tool at index 51
model_class = agent_tools[51].args_schema
try:
    # Attempt to print the source code of the Pydantic model class
    source_code = inspect.getsource(model_class)
    pprint(source_code)
except OSError:
    # Handle the case where the source code cannot be found
    pprint(f"Could not find the source code for the class {model_class.__name__}")

In [ ]:
agent_tools[51].func

In [ ]:
import inspect
from pprint import pprint

function = inspect.getsource(agent_tools[60].func)
pprint(function)

In [ ]:
from composio_langchain.toolset import ComposioToolSet

composio_toolset = ComposioToolSet(api_key=os.getenv('COMPOSIO_API_KEY'))
composio_toolset

In [ ]:
dir(composio_toolset)